In [4]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from keras.utils.np_utils import to_categorical
import random, shutil
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model







# img,labels= next(train_batch)
# print(img.shape)





In [13]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1. / 255), shuffle=True, batch_size=1, target_size=(24, 24),
              class_mode='categorical'):
    return gen.flow_from_directory(dir, batch_size=batch_size, shuffle=shuffle, color_mode='grayscale',
                                   class_mode=class_mode, target_size=target_size)

In [15]:
BS = 32
TS = (24, 24)
train_batch = generator('data/train', shuffle=True, batch_size=BS, target_size=TS)
valid_batch = generator('data/test', shuffle=True, batch_size=BS, target_size=TS)
SPE = len(train_batch.classes) // BS
VS = len(valid_batch.classes) // BS
print(SPE, VS)

Found 200 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
6 0


In [17]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24, 24, 1)),
    MaxPooling2D(pool_size=(1, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),
    # 32 convolution filters used each of size 3x3
    # again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1, 1)),

    # 64 convolution filters used each of size 3x3
    # choose the best features via pooling

    # randomly turn neurons on and off to improve convergence
    Dropout(0.25),
    # flatten since too many dimensions, we only want a classification output
    Flatten(),
    # fully connected to get all relevant data
    Dense(128, activation='relu'),
    # one more dropout for convergence' sake :)
    Dropout(0.5),
    # output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [19]:
model.fit_generator(train_batch, validation_data=valid_batch, epochs=15, steps_per_epoch=SPE, validation_steps=VS)


Epoch 1/15
6/6 [==============================] - 3s 577ms/step - loss: 0.7222 - accuracy: 0.5179
Epoch 2/15
6/6 [==============================] - 1s 151ms/step - loss: 0.6911 - accuracy: 0.5714
Epoch 3/15
6/6 [==============================] - 1s 146ms/step - loss: 0.6496 - accuracy: 0.5476
Epoch 4/15
6/6 [==============================] - 1s 150ms/step - loss: 0.6349 - accuracy: 0.6310
Epoch 5/15
6/6 [==============================] - 1s 159ms/step - loss: 0.6592 - accuracy: 0.5990
Epoch 6/15
6/6 [==============================] - 1s 151ms/step - loss: 0.5964 - accuracy: 0.6250
Epoch 7/15
6/6 [==============================] - 1s 148ms/step - loss: 0.5042 - accuracy: 0.7321
Epoch 8/15
6/6 [==============================] - 1s 151ms/step - loss: 0.4543 - accuracy: 0.7857
Epoch 9/15
6/6 [==============================] - 1s 147ms/step - loss: 0.3956 - accuracy: 0.82140s - loss: 0.3844 - accuracy: 0.86
Epoch 10/15
6/6 [==============================] - 1s 179ms/step - loss: 0.3715 - ac

In [16]:
model.save('cnnCat2.h5', overwrite=True)